In [1]:
import torch
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
import torchvision
import glob

In [2]:
classes = ['f','j','k','l','m','n','o','x','y','z']

transformer = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],
                        [0.5,0.5,0.5])
])

test_path = 'data_letters/test'

dataloader = DataLoader(
    torchvision.datasets.ImageFolder(test_path, transform=transformer),
    batch_size = 256,
    shuffle = False
)

In [3]:
class Task1Net(nn.Module):
    def __init__(self, num_classes=10):
        super(Task1Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(num_features=12)
        self.relu1 = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.dropout = nn.Dropout(0.5)
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=20, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.conv3 = nn.Conv2d(in_channels=20, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(num_features=32)
        self.relu3 = nn.ReLU()
        self.fc = nn.Linear(in_features=32*32*32, out_features=num_classes)
        
    def forward(self, input):
        output = self.conv1(input)
        output = self.bn1(output)
        output = self.relu1(output)
        output = self.pool(output)
        output = self.dropout(output)
        output = self.conv2(output)
        output = self.relu2(output)
        output = self.conv3(output)
        output = self.bn3(output)
        output = self.relu3(output)
        output = output.view(-1, 32*32*32)
        output = self.fc(output)
        return output

In [4]:
# model = Task1Net(num_classes=10)
model = torchvision.models.resnet34(pretrained=False)
inchannel = model.fc.in_features
model.fc = nn.Linear(inchannel, 10)
best_model = torch.load('model/task1.model')
model.load_state_dict(best_model, strict=False)
count = len(glob.glob(test_path + '/**/*.bmp'))
model.eval()

sum_accuracy = 0.0
for i, (images, labels) in enumerate(dataloader):
    if torch.cuda.is_available():
        images = Variables(images.cuda())
        labels = Variables(images.cuda())
    outputs = model(images)
    _,prediction = torch.max(outputs.data,1)
    sum_accuracy += int(torch.sum(prediction==labels.data))

accuracy = sum_accuracy / count

print('Accuracy: ' + str(round(accuracy*100,2)) + '%')

Accuracy: 83.04%
